In [1]:
import cv2
import dlib
import numpy as np
import os
import pandas as pd # Import pandas for CSV handling

# Load detector and predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(
    "models/shape_predictor_68_face_landmarks.dat/shape_predictor_68_face_landmarks.dat"
)

# Define the root directory for your emotion folders
root_input_dir = "archive_3"
root_output_dir = "processed_faces_final"

# Define the scaling factor for upscaling images
scale_factor = 6

# Create the root output directory if it doesn't exist
os.makedirs(root_output_dir, exist_ok=True)

# List of emotion folders (assuming these are the names of your subfolders)
emotion_folders = ["anger", "contempt", "disgust", "happy", "sadness", "fear", "surprise"]

# List to store all landmark data for CSV export
all_landmarks_data = []

print("Starting facial landmark detection and processing for ALL photos...")
print("Output will be split into '_with_photo' and '_without_photo' folders.")
print("A comprehensive CSV file with landmark coordinates will also be generated.")


for emotion in emotion_folders:
    input_emotion_path = os.path.join(root_input_dir, emotion)
    
    # Define the new output subfolders for each emotion
    output_with_photo_path = os.path.join(root_output_dir, emotion, f"{emotion}_with_photo")
    output_without_photo_path = os.path.join(root_output_dir, emotion, f"{emotion}_without_photo")

    # Create these new output subdirectories if they don't exist
    os.makedirs(output_with_photo_path, exist_ok=True)
    os.makedirs(output_without_photo_path, exist_ok=True)
    
    print(f"\nProcessing emotion: {emotion}...")

    # Iterate through each image in the emotion folder
    for filename in os.listdir(input_emotion_path):
        # Filter for image files
        if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
            image_path = os.path.join(input_emotion_path, filename)
            image = cv2.imread(image_path)

            if image is None:
                print(f"Warning: Could not read image {image_path}. Skipping.")
                continue # Continue to the next file if image cannot be read
                
            print(f"  Processing file: {filename}")

            # Upscale the image
            upscaled_image = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)
            gray = cv2.cvtColor(upscaled_image, cv2.COLOR_BGR2GRAY)

            # Detect faces
            faces = detector(gray)

            if len(faces) > 0:
                # Process the first detected face (you can modify this if you need to process all faces)
                face = faces[0]
                landmarks = predictor(gray, face)

                # --- Collect landmark data for CSV ---
                landmark_row = {'emotion': emotion, 'filename': filename}
                for n in range(68):
                    x = landmarks.part(n).x
                    y = landmarks.part(n).y
                    landmark_row[f'landmark_{n}_x'] = x
                    landmark_row[f'landmark_{n}_y'] = y
                all_landmarks_data.append(landmark_row)
                # --- End of CSV data collection ---

                # --- Prepare images for both output types ---
                # 1. Image with original photo and landmarks (for _with_photo folder)
                drawn_image_with_photo = upscaled_image.copy() 

                # 2. Blank image for landmarks only (for _without_photo folder)
                # Create a black background image of the same size as the upscaled image
                drawn_image_without_photo = np.zeros(upscaled_image.shape, dtype=np.uint8) 

                # Get position of landmark 31 (index 30 in 0-based indexing)
                nose_x = landmarks.part(30).x
                nose_y = landmarks.part(30).y

                # Draw quadrant lines centered at landmark 31 (only on image WITH photo)
                img_h, img_w = drawn_image_with_photo.shape[:2]
                cv2.line(drawn_image_with_photo, (nose_x, 0), (nose_x, img_h), (0, 255, 0), 1)
                cv2.line(drawn_image_with_photo, (0, nose_y), (img_w, nose_y), (0, 255, 0), 1)

                drawn_index = 1
                seen = set()

                for n in range(68):
                    x = landmarks.part(n).x
                    y = landmarks.part(n).y

                    if (x, y) in seen:
                        continue
                    seen.add((x, y))

                    # Determine circle color and size
                    circle_color = (0, 0, 255) # Red
                    circle_radius = 5 if n == 30 else 2
                    fill_type = -1 # Filled circle

                    # Determine text properties
                    text_font = cv2.FONT_HERSHEY_SIMPLEX
                    text_scale = 0.8 if n == 30 else 0.35
                    text_color = (0, 0, 0) # Black
                    text_thickness = 2 if n == 30 else 1
                    
                    # Offset for label (re-using your logic for consistency)
                    offset_x, offset_y = -6, 10
                    if n in [37, 38, 39, 43, 44, 45]:
                        offset_y = -10
                    elif n == 49:
                        offset_x = -20
                        offset_y = 6
                    elif n == 50:
                        offset_x = 0
                        offset_y = -8
                    elif n == 65:
                        offset_x = -14
                        offset_y = 10
                    elif 48 <= n <= 54:
                        offset_y = -8
                    elif 55 <= n <= 59:
                        offset_y = 10
                    elif 60 <= n <= 64:
                        offset_y = -8
                    elif 65 <= n <= 67:
                        offset_y = 10
                    
                    # Draw on the image WITH photo
                    cv2.circle(drawn_image_with_photo, (x, y), circle_radius, circle_color, fill_type)
                    cv2.putText(drawn_image_with_photo, str(drawn_index), (x + offset_x, y + offset_y),
                                text_font, text_scale, text_color, text_thickness, cv2.LINE_AA)

                    # Draw on the image WITHOUT photo (landmarks only)
                    # Use white color (255, 255, 255) for landmarks on black background for visibility
                    cv2.circle(drawn_image_without_photo, (x, y), circle_radius, (255, 255, 255), fill_type) 
                    cv2.putText(drawn_image_without_photo, str(drawn_index), (x + offset_x, y + offset_y),
                                text_font, text_scale, (255, 255, 255), text_thickness, cv2.LINE_AA) # White text

                    drawn_index += 1

                # Save the processed images to their respective folders
                output_image_path_with_photo = os.path.join(output_with_photo_path, filename)
                cv2.imwrite(output_image_path_with_photo, drawn_image_with_photo)
                
                output_image_path_without_photo = os.path.join(output_without_photo_path, filename)
                cv2.imwrite(output_image_path_without_photo, drawn_image_without_photo)
                
            else:
                print(f"  No face detected in {filename} from {emotion} folder.")
            
            # --- The 'break' statement for testing has been REMOVED here ---
            # This ensures all images in the folder are processed.

print("\nAll emotion folders and photos processed successfully!")

# --- Save all collected landmark data to a single CSV file ---
if all_landmarks_data:
    df = pd.DataFrame(all_landmarks_data)
    csv_output_path = os.path.join(root_output_dir, "facial_landmarks_data.csv")
    df.to_csv(csv_output_path, index=False)
    print(f"All facial landmark coordinates saved to: {csv_output_path}")
else:
    print("No facial landmark data was collected (no faces detected or no images processed).")



Starting facial landmark detection and processing for ALL photos...
Output will be split into '_with_photo' and '_without_photo' folders.
A comprehensive CSV file with landmark coordinates will also be generated.

Processing emotion: anger...
  Processing file: S010_004_00000017.png
  Processing file: S010_004_00000018.png
  Processing file: S010_004_00000019.png
  Processing file: S011_004_00000019.png
  Processing file: S011_004_00000020.png
  Processing file: S011_004_00000021.png
  Processing file: S014_003_00000028.png
  Processing file: S014_003_00000029.png
  Processing file: S014_003_00000030.png
  Processing file: S022_005_00000030.png
  Processing file: S022_005_00000031.png
  Processing file: S022_005_00000032.png
  Processing file: S026_003_00000013.png
  Processing file: S026_003_00000014.png
  Processing file: S026_003_00000015.png
  Processing file: S028_001_00000022.png
  Processing file: S028_001_00000023.png
  Processing file: S028_001_00000024.png
  Processing file: 

In [ ]:
import cv2
import dlib
import numpy as np
import math

# Load detector and predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(
    r"C:\Users\sahug\Downloads\shape_predictor_68_face_landmarks.dat\shape_predictor_68_face_landmarks.dat"
)

# Load and stretch image
image_path = r"C:\Users\sahug\Downloads\archive_3\contempt\S158_002_00000011.png"
image = cv2.imread(image_path)
scale_x = 12  # Horizontal stretch
scale_y = 12  # Vertical stretch
image = cv2.resize(image, (0, 0), fx=scale_x, fy=scale_y)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect face
faces = detector(gray)

def rotate_image(img, angle, center=None, scale=1.0):
    (h, w) = img.shape[:2]
    if center is None:
        center = (w // 2, h // 2)
    rot_mat = cv2.getRotationMatrix2D(center, angle, scale)
    return cv2.warpAffine(img, rot_mat, (w, h))

# Process first face
if len(faces) > 0:
    face = faces[0]
    landmarks = predictor(gray, face)

    # Eye centers
    left_eye_pts = [landmarks.part(n) for n in range(36, 42)]
    right_eye_pts = [landmarks.part(n) for n in range(42, 48)]
    left_eye_center = np.mean([(pt.x, pt.y) for pt in left_eye_pts], axis=0)
    right_eye_center = np.mean([(pt.x, pt.y) for pt in right_eye_pts], axis=0)

    # Rotate image
    dy = right_eye_center[1] - left_eye_center[1]
    dx = right_eye_center[0] - left_eye_center[0]
    angle = math.degrees(math.atan2(dy, dx))
    rotated_image = rotate_image(image, angle)
    rotated_gray = cv2.cvtColor(rotated_image, cv2.COLOR_BGR2GRAY)

    # Detect again on rotated
    rotated_faces = detector(rotated_gray)
    if len(rotated_faces) > 0:
        rotated_landmarks = predictor(rotated_gray, rotated_faces[0])

        drawn_index = 1
        seen = set()

        # Get position of landmark 31 (index 30 in 0-based indexing)
        nose_x = rotated_landmarks.part(30).x
        nose_y = rotated_landmarks.part(30).y

        # Draw quadrant lines centered at landmark 31
        img_h, img_w = rotated_image.shape[:2]
        cv2.line(rotated_image, (nose_x, 0), (nose_x, img_h), (0, 255, 0), 1)
        cv2.line(rotated_image, (0, nose_y), (img_w, nose_y), (0, 255, 0), 1)

        for n in range(68):
            x = rotated_landmarks.part(n).x
            y = rotated_landmarks.part(n).y

            if (x, y) in seen:
                continue
            seen.add((x, y))

            # Special highlight for landmark 31
            if n == 30:
                cv2.circle(rotated_image, (x, y), 5, (0, 0, 255), -1)
                cv2.putText(rotated_image, str(drawn_index), (x - 12, y - 12),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)
            else:
                # Draw standard landmark
                cv2.circle(rotated_image, (x, y), 2, (0, 0, 255), -1)

                # Offset for label
                offset_x, offset_y = -6, 10

                if n in [37, 38, 39, 43, 44, 45]:
                    offset_y = -10
                elif n == 49:
                    offset_x = -20
                    offset_y = 6
                elif n == 50:
                    offset_x = 0
                    offset_y = -8
                elif n == 65:
                    offset_x = -14
                    offset_y = 10
                elif 48 <= n <= 54:
                    offset_y = -8
                elif 55 <= n <= 59:
                    offset_y = 10
                elif 60 <= n <= 64:
                    offset_y = -8
                elif 65 <= n <= 67:
                    offset_y = 10

                # Draw label
                cv2.putText(rotated_image, str(drawn_index), (x + offset_x, y + offset_y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 0), 1, cv2.LINE_AA)

            drawn_index += 1

        # Show and save result
        cv2.imshow("Rotated Face Landmarks with Quadrants", rotated_image)
        cv2.imwrite("quadrants_landmarks_rotated.png", rotated_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Face not detected after rotation.")
else:
    print("No face detected.")


Starting facial landmark detection and processing for ALL photos...
Output images will be split into '_with_photo' and '_without_photo' folders.
A comprehensive CSV file with landmark coordinates will also be generated.

Processing emotion: anger...
  Processing file: S010_004_00000017.png
  Processing file: S010_004_00000018.png
  Processing file: S010_004_00000019.png
  Processing file: S011_004_00000019.png
  Processing file: S011_004_00000020.png
  Processing file: S011_004_00000021.png
  Processing file: S014_003_00000028.png
  Processing file: S014_003_00000029.png
  Processing file: S014_003_00000030.png
  Processing file: S022_005_00000030.png
  Processing file: S022_005_00000031.png
  Processing file: S022_005_00000032.png
  Processing file: S026_003_00000013.png
  Processing file: S026_003_00000014.png
  Processing file: S026_003_00000015.png
  Processing file: S028_001_00000022.png
  Processing file: S028_001_00000023.png
  Processing file: S028_001_00000024.png
  Processing